In [1]:
import numpy as np
import tensorflow as tf
import torch
tf.get_logger().setLevel('ERROR')
!curl -s https://raw.githubusercontent.com/sivel/speedtest-cli/master/speedtest.py | python -
     


2023-10-08 23:27:29.061168: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-10-08 23:27:29.120509: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-08 23:27:29.358877: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-10-08 23:27:29.358905: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-10-08 23:27:29.360397: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to regi

Retrieving speedtest.net configuration...
Testing from VNPT (113.161.73.9)...
Retrieving speedtest.net server list...
Selecting best server based on ping...
Hosted by FPT Telecom (Ho Chi Minh City) [9.23 km]: 14.879 ms
Testing download speed................................................................................
Download: 11.14 Mbit/s
Testing upload speed......................................................................................................
Upload: 9.42 Mbit/s


In [2]:
TRAIN_PATH = 'VLSP2018-SA-train-dev-test/csv/train.csv'
VAL_PATH = 'VLSP2018-SA-train-dev-test/csv/dev.csv'
TEST_PATH = 'VLSP2018-SA-train-dev-test/csv/test.csv'


In [3]:
import pandas as pd
def read_csv(url):
    df = pd.read_csv(url)

    X = df.pop('review')
    y = df.replace({np.nan: 0, 
                    'negative': 1, 
                    'neutral': 2, 
                    'positive': 3}).astype(np.uint8)

    print('X.shape:', X.shape, 'y.shape:', y.shape)
    return X, y

Xtrain, ytrain = read_csv(TRAIN_PATH)
Xdev,   ydev   = read_csv(VAL_PATH)
Xtest,  ytest  = read_csv(TEST_PATH)

X.shape: (2961,) y.shape: (2961, 12)
X.shape: (1290,) y.shape: (1290, 12)
X.shape: (500,) y.shape: (500, 12)


In [4]:
from functools import partial
import demoji
from flashtext import KeywordProcessor
from sklearn.base import BaseEstimator, TransformerMixin
import unicodedata

HASHTAG = 'hashtag'

class TextCleanerBase(BaseEstimator, TransformerMixin):
    def __init__(self):
        super().__init__()

        # Create preprocessing function
        self.normalize_unicode = partial(unicodedata.normalize, 'NFC')
            
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        if not isinstance(X, pd.Series):
            X = pd.Series(X)

        return X.apply(str.lower) \
                .apply(remove_emojis) \
                .apply(self.normalize_unicode)

def remove_emojis(text):
    return demoji.replace(text, '')

In [5]:
aspects = ['FOOD#PRICES',
           'FOOD#QUALITY',
           'FOOD#STYLE&OPTIONS',
           'DRINKS#PRICES',
           'DRINKS#QUALITY',
           'DRINKS#STYLE&OPTIONS',
           'RESTAURANT#PRICES',
           'RESTAURANT#GENERAL',
           'RESTAURANT#MISCELLANEOUS',
           'SERVICE#GENERAL',
           'AMBIENCE#GENERAL',
           'LOCATION#GENERAL']

sentiments = ['-', 'o', '+']

def mo2ml(y):
    """Convert multi-output to multi-label data
    """
    newcols = [f'{a} {s}' for a in aspects for s in sentiments]

    nrows, ncols = len(y), len(newcols)
    ml = pd.DataFrame(np.zeros((nrows, ncols), dtype='bool'),
                      columns=newcols)
    
    for i, a in enumerate(aspects):
        for j in range(1, 4):
            indices = y[a] == j
            ml.iloc[indices, i * 3 + j - 1] = True

    return ml

In [6]:
def mo2df(y):
    if isinstance(y, pd.DataFrame):
        return y
    return pd.DataFrame(y, columns=aspects)

In [7]:
cleaner_base  = TextCleanerBase()

xtrain_basecl = cleaner_base.transform(Xtrain)
xdev_basecl   = cleaner_base.transform(Xdev)
xtest_basecl  = cleaner_base.transform(Xtest)

ytrain_b  = ytrain.copy()
ydev_b    = ydev  .copy()
ytest_b   = ytest .copy()

ytrain_ml = mo2ml(ytrain)
ydev_ml   = mo2ml(ydev)
ytest_ml  = mo2ml(ytest)

xtrain_basecl

0       _ ảnh chụp từ hôm qua, đi chơi với gia đình và...
1       _hương vị thơm ngon, ăn cay cay rất thích, nêm...
2       - 1 bàn tiệc hoành tráng 3 đứa ăn no muốn tắt ...
3       - các bạn nhìn cái chảo này có to không - trên...
4       - cháo: có nhiều hương cho các bạn chọn, nhưng...
                              ...                        
2956                                 y hệt vị đà lạt luôn
2957    yaourt trái cây mát lạnh, có thêm viên kem ở t...
2958               zumi.... zumi lễ vẫn bán nhé mọi người
2959    set này có 2 tầng bánh và 1 ấm trà mà chỉ có 1...
2960    lạnh trời thế này mà ngồi ăn chả cá lăng xèo x...
Name: review, Length: 2961, dtype: object

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(ngram_range=(1, 3),
                             min_df=2, max_df=0.9)

# x data using basic clean up class and basic features extrator
xtrain_basecl_basef = vectorizer.fit_transform(xtrain_basecl)
xdev_basecl_basef   = vectorizer.transform(xdev_basecl)
xtest_basecl_basef  = vectorizer.transform(xtest_basecl)


In [9]:
from sklearn.metrics import f1_score, classification_report


def quick_f1(y_true, y_pred):
    y_pred = mo2ml(mo2df(y_pred))
    return round(f1_score(y_true, y_pred, average='micro', zero_division=0), 4)

def evaluate(model, X, y, average='micro'):
    yb_true  = mo2ml(y)

    yb_pred  = mo2df(model.predict(X))
    yb_pred  = mo2ml(yb_pred)

    return classification_report(yb_true, yb_pred, zero_division=0)

In [10]:
from sklearn.linear_model import LogisticRegression
import optuna
from optuna.samplers import TPESampler
from sklearn.multioutput import MultiOutputClassifier as MOC

def callback(study, trial):
    if study.best_trial.number == trial.number:
        study.set_user_attr(key='best_model', value=trial.user_attrs['model'])

def logistic_objective(trial):
    params = dict(
        class_weight=trial.suggest_categorical('class_weight', ['balanced', None]),
        C=trial.suggest_float('C', 1e-5, 20),
        random_state=5,
    )    

    clf = MOC(LogisticRegression(solver='sag', max_iter=200, **params))
    clf.fit(xtrain_basecl_basef, ytrain_b)
    trial.set_user_attr(key="model", value=clf)

    y_pred = clf.predict(xdev_basecl_basef)
    return quick_f1(ydev_ml, y_pred)

sampler = TPESampler(seed=22)
logistic_study = optuna.create_study(sampler=sampler, direction='maximize')
logistic_study.optimize(logistic_objective, n_trials=5, callbacks=[callback])


clf4 = logistic_study.user_attrs['best_model']

print(evaluate(clf4, xtest_basecl_basef, ytest_b))

print('train:', quick_f1(ytrain_ml, clf4.predict(xtrain_basecl_basef)))
print('dev:  ', quick_f1(ydev_ml  , clf4.predict(xdev_basecl_basef)))
print('test:', quick_f1(ytest_ml  , clf4.predict(xtest_basecl_basef)))

print(clf4.estimators_[0].get_params())
print(logistic_study.best_params)

/home/aligator/.local/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[I 2023-10-08 23:28:12,843] A new study created in memory with name: no-name-d91e8dce-0c08-463f-ba11-cb92da3a70a4
[I 2023-10-08 23:28:17,202] Trial 0 finished with value: 0.6548 and parameters: {'class_weight': None, 'C': 8.41076650090714}. Best is trial 0 with value: 0.6548.
/home/aligator/.local/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/aligator/.local/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/aligator/.local/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:350: Convergenc

              precision    recall  f1-score   support

           0       0.46      0.21      0.29        28
           1       0.54      0.53      0.53       175
           2       0.47      0.63      0.54       128
           3       0.00      0.00      0.00        11
           4       0.49      0.47      0.48        43
           5       0.88      0.93      0.91       403
           6       1.00      0.06      0.12        16
           7       0.22      0.04      0.06        53
           8       0.74      0.95      0.83       334
           9       0.00      0.00      0.00         3
          10       0.50      0.09      0.15        45
          11       0.25      0.07      0.11        28
          12       0.00      0.00      0.00         6
          13       0.00      0.00      0.00        11
          14       0.56      0.41      0.47        54
          15       0.00      0.00      0.00         1
          16       0.00      0.00      0.00         4
          17       0.53    

In [11]:
test_corpus = [
   'Quán này mình biết cũng khá lâu rồi, lâu lâu cũng hay ghé ăn lắm, mà lần nào lại thấy cũng đông nghẹt. Nằm trong khu chung cư nên không gian rất thích, tuy bàn chủ yếu ở ngoài trời nhưng không bị khói bụi, rất mát mẻ, thích hợp ngồi ăn uống tám chuyện buổi tối lắm, chắc vì vậy mà mấy bạn trẻ ghé đây ăn rất nhiều. Đặc biệt món súp cua ở đây khá hot, tới trễ là hết liền. Bữa mình đi tính ăn, mới hơn 7h vậy mà lại hết sạch :(  Hết súp cua nên kêu mì ốc với bánh flan ăn nè  Mì xào ốc tỏi - 45k   Mì ở đây là mì gói thì phải, ăn sợi mì cũng bình thường nhưng nêm nếm rất ngon, có tỏi cháy làm rất thơm ngon, mùi vị đậm đà, khỏi chan nước mắm vô nghen.  Ốc tỏi xắt miếng nhỏ, ăn thấy tươi và ngon, nhưng thấy hơi ít. Giá này thấy cũng hơi mắc nghen!  Bánh flan kêu 2 phần flan thường với flan rau câu dừa - 10k/dĩa   Bánh flan ở đây không có nhiều loại, chỉ có nhiêu đây hà   Phần rau câu dừa này có flan với cái dừa bên trong nha, ăn vừa dai dai sật sật, vừa béo thơm ngon lắm  Flan ở đây làm cũng ngon lắm nè, bánh ăn béo mịn, ngọt vừa phải, làm theo kiểu truyền thống thôi nên không có đủ loại, và chỉ rưới cà phê lên chứ cũng không có topping gì hết nhưng mình ăn thấy thích lắm  Không gian bên ngoài nè, nằm bên mé chung cư  Ở giữa là bãi cỏ công viên nên không khí khá mát mẻ và trong lành, bàn kê ra khắp nơi, nhằm khi để cả lên cỏ',
]
text = vectorizer.transform(test_corpus)


In [12]:
clf4.predict_proba(text)

[array([[0.04809804, 0.1004057 , 0.76526105, 0.08623521]]),
 array([[0.00561806, 0.00952508, 0.16233857, 0.82251829]]),
 array([[0.02285304, 0.04970076, 0.07852333, 0.84892287]]),
 array([[0.78947172, 0.01868046, 0.11601795, 0.07582988]]),
 array([[0.72580452, 0.00627059, 0.03864078, 0.22928411]]),
 array([[7.49719304e-01, 2.50261839e-03, 1.03865657e-04, 2.47674212e-01]]),
 array([[0.76936978, 0.03075209, 0.08141528, 0.11846284]]),
 array([[0.07298189, 0.01261912, 0.02712856, 0.88727043]]),
 array([[0.87847046, 0.07623925, 0.03264614, 0.01264414]]),
 array([[0.38973684, 0.13927879, 0.20793398, 0.26305039]]),
 array([[0.07102182, 0.07911105, 0.31715705, 0.53271008]]),
 array([[0.50098124, 0.07774878, 0.09138668, 0.32988331]])]

In [15]:
from sklearn.pipeline import make_pipeline
import joblib

pipe = make_pipeline(TextCleanerBase(), vectorizer, clf4)
joblib.dump(pipe, 'pipe.joblib')



['pipe.joblib']